## <center>Ubuntu环境下ChatGLM3-6B安装办法


&emsp;&emsp;
大模型的部署整体而言并不复杂，官方通常会提供标准的部署流程。然而，许多人在实际部署过程中会遇到各种问题，导致部署难以成功。主要原因在于这个过程中涉及到大量依赖库的安装和更新，以及需要根据本地环境的安装情况及时调整代码逻辑。此外，部署还对硬件有一定的要求，因此整体而言，部署和使用仍然存在一定门槛。为此，我们特别整理了一份详细的ChatGLM3-6B模型部署教程，供大家参考和学习。

在本教程主要涉及了以下三个部分：
1. 大模型参数介绍
2. ChatGLM3-6B基本介绍
3. ChatGLM3-6B本地部署流程

# 1. 大模型参数介绍

当你第一次了解一些开源大模型的时候，往往会困惑于模型琳琅满目的开源版本，不知道自己应该如何选择合适的模型，这些模型的差异又是什么？以下是大模型的相关参数介绍以供为大家解答这方面的疑惑。

### 1.1.1 模型参数单位

在浏览模型信息的时候常常会看到类似："6b"、"13b"、"70b"等字样，这个术语通常指的是大型神经网络模型的参数数量。其中的 "b" 代表 "billion"，也就是十亿。表示模型中的参数量，每个参数用来存储模型的权重和偏差等信息。例如：

"6b" 意味着模型有大约 60 亿个参数。
"13b" 意味着模型有大约 130 亿个参数。
"70b" 意味着模型有大约 700 亿个参数。

### 1.1.2 模型参数精度

模型参数的精度通常指的是参数的数据类型，它决定了模型在内存中存储和计算参数时所使用的位数。以下是一些常见的模型参数精度及其含义，以及它们在内存中所占用的字节数。

- 单精度浮点数 (32位) - float32:   
  - 含义：单精度浮点数用于表示实数，具有较高的精度，适用于大多数深度学习应用。
  - 字节数：4字节（32位）
- 半精度浮点数 (16位) - float16:   
  - 含义：半精度浮点数用于表示实数，但相对于单精度浮点数，它的位数较少，因此精度稍低。然而，它可以在某些情况下显著减少内存占用并加速计算。
  - 字节数：2字节（16位）
- 双精度浮点数 (64位) - float64:   
  - 含义：双精度浮点数提供更高的精度，适用于需要更高数值精度的应用，但会占用更多的内存。
  - 字节数：8字节（64位）
- 整数 (通常为32位或64位) - int32, int64:   
  - 含义：整数用于表示离散的数值，可以是有符号或无符号的。在某些情况下，例如分类问题中的标签，可以使用整数数据类型来表示类别。   
  - 字节数：通常为4字节（32位）或8字节（64位）

模型参数精度的选择往往是一种权衡。**使用更高精度的数据类型可以提供更高的数值精度，但会占用更多的内存并可能导致计算速度变慢。相反，使用较低精度的数据类型可以节省内存并加速计算，但可能会导致数值精度损失。** 在实际应用中，选择模型参数的精度需要根据具体任务、硬件设备和性能要求进行权衡考虑。

实际上，通常情况下并没有标准的整数数据类型为int4或int8，因为这些整数数据类型不太常见，且在大多数计算机体系结构中没有直接支持。在计算机中，整数通常以字节为单位进行存储，所以int4表示一个4位的整数，int8表示一个8位的整数。

然而，近年来在深度学习领域中，出于模型压缩和加速的考虑，研究人员开始尝试使用较低位数的整数来表示模型参数。例如，一些研究工作中使用的int4、int8等整数表示法是通过 **量化（quantization）** 技术来实现的。

在量化技术中，int4和int8分别表示4位和8位整数。这些整数用于表示模型参数，从而减少模型在存储和计算时所需的内存和计算资源。量化是一种模型压缩技术，通过将浮点数参数映射到较低位数的整数，从而在一定程度上降低了模型的计算和存储成本。以下是对这两种精度的解释以及它们在内存中占用的字节数：

- int4 (4位整数):

  - 含义：int4使用4位二进制来表示整数。在量化过程中，浮点数参数将被映射到一个有限的范围内的整数，然后使用4位来存储这些整数。
  - 字节数：由于一个字节是8位，具体占用位数而非字节数，通常使用位操作存储。

- int8 (8位整数):

  - 含义：int8使用8位二进制来表示整数。在量化过程中，浮点数参数将被映射到一个有限的范围内的整数，然后使用8位来存储这些整数。
  - 字节数：1字节（8位）

在量化过程中，模型参数的值被量化为最接近的可表示整数，这可能会导致一些信息损失。因此，在使用量化技术时，需要平衡压缩效果和模型性能之间的权衡，并根据具体任务的需求来选择合适的量化精度。

### 1.1.3 模型文本后缀
开源模型的不同后缀通常指示了模型的特定版本或特化用途，反映了模型被调整或优化的方向。

- 无后缀模型：这通常指的是标准模型，未经过特殊优化或调整，可用于广泛的用途和基本的自然语言处理任务。

- Instruct模型：指令微调模型。在指令微调模型中输入的“指令”通常是清晰、明确的文本命令，用于告诉模型应该执行什么样的任务。这些指令旨在使模型理解并精确执行所需的操作，这些操作可以是生成文本、回答问题、提供信息、甚至进行复杂的数据分析等。

- chat模型：这类模型专为对话系统和聊天应用优化。它们经过训练以更好地处理交互式对话，可能包括更好的上下文保持能力和回复生成的连贯性。

- MoE模型：是指模型采用（混合专家 · Mixture of Experts）架构来部署的。这种方法的工作原理是将复杂的任务划分为更小、更易于管理的子任务，每个子任务都由专门的迷你模型或「专家」处理。

具体的文本类型后缀，我们可以在模型下对应的介绍页面中找到具体的意义。

chat类模型通常是我们最常选择的部署模型，尤其在尝试Agent开发的过程中，希望获得更好的交互式对话体验。

## 1.2 推理显存计算
模型推理（inference）是指在已经训练好的模型上对新的数据进行预测或分类。推理阶段通常比训练阶段要求更低的显存，因为不涉及梯度计算和参数更新等大量计算。以下是计算模型推理时所需显存的一些关键因素：

- 模型结构： 模型的结构包括层数、每层的神经元数量、卷积核大小等。较深的模型通常需要更多的显存，因为每一层都会产生中间计算结果。

- 输入数据： 推理时所需的显存与输入数据的尺寸有关。更大尺寸的输入数据会占用更多的显存。

- 批处理大小BatchSize： 批处理大小是指一次推理中处理的样本数量。较大的批处理大小可能会增加显存使用，因为需要同时存储多个样本的计算结果。

- 数据类型DataType： 使用的数据类型（如单精度浮点数、半精度浮点数）也会影响显存需求。较低精度的数据类型通常会减少显存需求。

- 中间计算： 在模型的推理过程中，可能会产生一些中间计算结果，这些中间结果也会占用一定的显存。


以 ChatGLM3-6B 为例

因为全精度模型参数是float32类型, 占用4个字节，粗略计算：1b(10亿)个模型参数，约占用4G显存(实际大小：10^9 * 4 / 1024^3 ~= 3.725 GB)，如果选择ChatGLM3-6B的参数量为6b，那么加载模型参数需要的显存为：3.725 * 6 ~= 22.35 GB

如果您的显存不足32GB，那么可以设置半精度的FP16/BF16来加载，每个参数只占2个字节，所需显存就直接减半，只需要约11GB。虽然模型效果会因精度损失而略微降低，但一般在可接受范围。

如果您的显存不足16GB，那么可以采用int8量化后，显存再减半，只需要约5.5GB，但是模型效果会更差一些。

如果您的显存不足8GB，那么只能采用int4量化，显存再减半，只需要约2.75GB，但是这个效果就差强人意了。

上述是加载模型到显存所需大小，在模型的推理过程中，可能会产生一些中间计算结果，这些中间结果也会占用一定的显存，所以显存大小不能刚好是参数量的大小。

## 1.4 GPU选择指南

### 1.4.1 学习实践

开源大模型的本地部署，不论是推理还是微调都离不开GPU这一硬件需求。相对而言，对CPU和内存的要求并不高。（尽管像GLM系列开源模型支持Intel CPU和Mac上运行，但部署操作十分繁琐且占用大量内存，因此大模型的相关应用探索，更推荐在GPU上进行。）CPU部分无论你是Intel还是AMD都可以，内存16G或者32G都可以，都十分灵活。

除硬件外，操作系统推荐使用Linux系统，相比Windows和MacOS，Linux对于大模型推理和微调有着更完善的支持，同时也是工业应用场景中使用最多的操作系统。

| 模型精度     | 7B 推理  | 7B训练（PEFT） | 最低配置显卡  | 显卡显存  | 硬件成本  | 租赁在线GPU成本  |
|-------------|------|------|------|------|------|------|
| FP32全精度   | 28GB | 40GB | 3090*2| 48GB| 8500*2| 30/h|
| FP16半精度   | 14GB | 20GB | 3090| 24GB| 8500| 30/h|
|             |      |      | 4080| 16GB| 7200| 25/h|
| Int8 精度    | 7GB  | 10GB | 2080Ti | 11GB | 2200 | 12/h |
|             |      |      | 3080Ti | 12GB | 4200 | 15/h |

我们当前的基本硬件配置如下：

![](https://fufanshare.oss-cn-beijing.aliyuncs.com/Data_Analysis_AI/Ch2_OpenSourceLLM/%E6%88%91%E4%BB%AC%E7%9A%84%E5%9F%BA%E6%9C%AC%E9%85%8D%E7%BD%AE.jpg)

### 1.4.2 工业应用

对于大模型工业级的实践要求，常见的消费及显卡就不能满足需求了。

| 显卡  | 目标市场       | 性能 | 应用场景                                    | 价格           |
|-------|----------------|------|--------------------------------------------|----------------|
| 4090  | 消费者         | 高   | 通用计算，图形渲染，高端游戏，4k/8k视频制作  | 14599（24G）    |
| T4    | 企业/AI推理    | 中   | AI推理，轻量级训练，图形渲染                | 8000（14G）    |
| A10   | 企业/图形      | 中   | 图形渲染，轻量级计算                        | 19000 (24G)    |
| V100  | 数据中心/AI    | 高   | 深度学习训练/推理，高性能计算              | 43000 (32G)    |
| A100  | 数据中心/AI    | 高   | 深度学习训练/推理，高性能计算              | 70000 (40G)    |
| H100  | 数据中心/AI    | 高   | 深度学习训练/推理，高性能计算，大数据分析  | 242000 (80G)   |
| A800  | 数据中心/AI    | 中高 | 深度学习推理，高性能计算，大数据分析        | 110000  (40G)  |


22年10月美国商务部限制 GPU 对华出口的算力不超过 4800 TOPS 和带宽不超过 600 GB/s，导致最强的 H100 和 A100 禁售。黄仁勋随后推出针对中国市场的 A800 和 H800，但随后23年10月，再次被禁止向中国出售。

- 对于本地个人研发项目，GeForce RTX 4090 等消费级 GPU 足以满足中等规模的需求。  
- 对于公司的大规模数据和复杂模型，推荐使用如 NVIDIA A100 的高性能 GPU。  
- 数据规模小时，可考虑预算内的 A10 或 T4 型号。  
- 如果没有双精度需求，追求性价比，可以选择把 4090 显卡搭建服务器使用。不适用于全量训练，可微调和推理。  

大模型训练需要高性能的计算机硬件来保证训练的效率和速度。建议选择具有高速的ECC或DDR5内存。现在训练时一般在 GLM 、 LLaMA 等大模型的基础上进行训练，国内会选择 A800/H800 x 8 的 GPU 配置，与此同时内存一般会选择相似大小以提升效率，常规选择 512 内存。

磁盘方面，大模型训练需要存储大规模的数据集和模型参数，因此需要足够的存储资源来保证数据能够快速地被读取和处理。建议选择具有大容量、高速的存储设备，如 SSD 或 NVMe 固态硬盘。一般 4T-8T 不等。

## 启动安装部署前的准备工作

- **操作系统要求**

&emsp;&emsp;首先看系统要求。目前开源的大模型都支持在Windows、Linux和Mac上部署运行。但在应用开发领域中，Linux 系统通常被优先选择而不是 Windows，主要原因是Linux 系统具有强大的包管理系统（如 apt, yum, pacman），允许开发者轻松安装、更新和管理软件包，这比 Windows 操作系统上的软件安装和管理更加方便快捷。同时Linux系统与多种编程语言和开发工具的兼容性较好，尤其是一些开源工具，仅支持在Linux系统上使用。整体来看，在应用运行方面对硬件的要求较低，且在处理多任务时表现出色，所以被广泛认为是一个非常稳定和可靠的系统，特别是对于服务器和长时间运行的应用。

&emsp;&emsp;Linux 操作系统有许多不同的发行版，每种发行版都有其特定的特点和用途，如CentOS、Ubuntu和Debian等。 CentOS 是一种企业级的 Linux 发行版，以稳定性和安全性著称。它是 RHEL（Red Hat Enterprise Linux）的免费替代品，与 RHEL 完全兼容，适用于服务器和企业环境。而Ubuntu，是最受欢迎的 Linux 发行版之一，其优势就是对用户友好和很强的易用性，其图形化界面都适合大部分人的习惯。

&emsp;&emsp;所以，在实践大模型时，强烈建议大家使用Ubuntu系统。同时，本教程也是针对Ubuntu 22.04 桌面版系统来进行ChatGLM3-6B模型的部署和运行的。

- **硬件配置要求**

&emsp;&emsp;其次，关于硬件的需求，ChatGLM3-6B支持GPU运行（需要英伟达显卡）、CPU运行以及Apple M系列芯片运行。其中GPU运行需要至少6GB以上显存（4Bit精度运行模式下），而CPU运行则需要至少32G的内存。由于Apple M系列芯片是统一内存架构，因此需要13G内存即可运行。其中CPU运行模式下内存占用过大且运行效率较低，因此我们也强调过，GPU模式部署才能有效的进行大模型的学习实践。

&emsp;&emsp;在本教程中，我们将重点讲解如何配置GPU环境来部署运行ChatGLM3-6B模型。

&emsp;&emsp;基于上述两方面的原因，我们在上期内容详细介绍了如何部署一个纯净的Ubuntu 22.04双系统。本期内容就在这样的环境基础上，安装必要的大模型运行依赖环境，并实际部署、运行及使用ChatGLM3-6B模型。

&emsp;&emsp;在开始之前，请大家确定当前使用的硬件环境满足ChatGLM3-6B模型本地化运行的官方最低配置需求：

<div align=center><img src="https://snowball101.oss-cn-beijing.aliyuncs.com/img/202401021814642.png" width=60%></div>

&emsp;&emsp;如果配置满足需求，接下来我们就一步一步执行本地化部署ChatGLM3-6B模型。本期内容将首先介绍ChatGLM3-6B模型在Ubuntu 22.04系统下单显卡部署流程，更加贴合实际工业生产的多卡推理部署办法将在未来的课程中展开讲述。


# 2. ChatGLM3-6B基本介绍

&emsp;&emsp;ChatGLM3 是智谱AI和清华大学 KEG 实验室在2023年10月27日联合发布的开源的、支持中英双语的对话语言模型。ChatGLM3-6B 是 ChatGLM3 系列中的开源模型具有62亿个参数，其可以免费下载，以及免费的商业化使用。


&emsp;&emsp;该模型在保留了前两代模型对话流畅、部署门槛低等众多优秀特性的基础上，ChatGLM3-6B 引入了如下特性：[ChatGLM 3 GitHub](https://github.com/THUDM/ChatGLM3)

1. **更强大的基础模型：** ChatGLM3-6B 的基础模型 ChatGLM3-6B-Base 采用了更多样的训练数据、更充分的训练步数和更合理的训练策略。在语义、数学、推理、代码、知识等不同角度的数据集上测评显示，在44个中英文公开数据集测试中处于国内模型的第一位。**ChatGLM3-6B-Base 具有在 10B 以下的基础模型中最强的性能**。
2. **更完整的功能支持：** ChatGLM3-6B 采用了全新设计的 [Prompt 格式](https://github.com/THUDM/ChatGLM3/blob/main/PROMPT.md)，除正常的多轮对话外。同时原生支持[工具调用](https://github.com/THUDM/ChatGLM3/blob/main/tool_using/README.md)（Function Call）、代码执行（Code Interpreter）和 Agent 任务等复杂场景。
3. **更全面的开源序列：** 除了对话模型 [ChatGLM3-6B](https://huggingface.co/THUDM/chatglm3-6b) 外，还开源了基础模型 [ChatGLM3-6B-Base](https://huggingface.co/THUDM/chatglm3-6b-base)、长文本对话模型 [ChatGLM3-6B-32K](https://huggingface.co/THUDM/chatglm3-6b-32k)。以上所有权重对学术研究**完全开放**，在填写[问卷](https://open.bigmodel.cn/mla/form)进行登记后**亦允许免费商业使用**。

<div align=center><img src="https://snowball101.oss-cn-beijing.aliyuncs.com/img/202401021645001.png" width=60%></div>

&emsp;&emsp;性能层面，ChatGLM3-6B在10B范围内性能表现出色，推理能力接近GPT-3.5；功能层面，ChatGLM3-6B具有多模态功能、代码解释器功能、联网功能以及Agent优化功能四项核心功能，是性能非常不错的小体量开源大模型。不过由于 ChatGLM3-6B 模型规模较小，且模型受概率随机性因素影响，无法保证输出内容的准确。同时模型的输出容易被用户的输入误导。想要获得较理想的效果还需要微调以及提示工程等方法的强化，

<div align=center><img src="https://snowball101.oss-cn-beijing.aliyuncs.com/img/202401021455839.png" width=40%></div>

&emsp;&emsp;AI Agent（人工智能代理）是一个能够自主执行任务或达成目标的系统或程序，能够围绕复杂问题进行任务拆解，规划多步执行步骤；能够实时围绕自动编写的代码进行debug；能够根据人类意见反馈修改答案，实时积累修改对话，并进行阶段性微调等等，具有很强的决策和执行能力。那ChatGLM3-6B模型开放的Function calling能力，是大语言模型推理能力和复杂问题处理能力的核心体现，是本次ChatGLM 3模型最为核心的功能迭代，也是ChatGLM 3模型性能提升的有力证明。

<div align=center><img src="https://snowball101.oss-cn-beijing.aliyuncs.com/img/202401021652206.png" width=60%></div>

&emsp;&emsp;相关的信息获取方途径

- 官方网站：https://www.zhipuai.cn/

- 智谱清言：https://chatglm.cn

- API开放平台：https://bigmodel.cn/

- Github仓库：https://github.com/THUDM/ChatGLM-6B

&emsp;&emsp;开源模型列表：

| **模型**             | **介绍**                                                     | **上下文token数** | **代码链接**                                                 | **模型权重下载链接**                                         |
| -------------------- | ------------------------------------------------------------ | ----------------- | ------------------------------------------------------------ | :----------------------------------------------------------- |
| **ChatGLM3-6B**      | **第三代** **ChatGLM 对话****模型****。**ChatGLM3-6B 采用了全新设计的 Prompt 格式，除正常的多轮对话外。同时原生支持工具调用（Function Call）、代码执行（Code Interpreter）和 Agent 任务等复杂场景。 | 8K                | [Huggingface](https://huggingface.co/THUDM/chatglm3-6b) \| [魔搭社区](https://modelscope.cn/models/ZhipuAI/chatglm3-6b) \| [始智社区](https://www.wisemodel.cn/models/ZhipuAI/chatglm3-6b) \| [Swanhub](https://swanhub.co/ZhipuAI/chatglm3-6b) \| [启智社区](https://openi.pcl.ac.cn/Zhipu.AI/ChatGLM3/modelmanage/model_readme_tmpl?name=chatglm3-6b) )                | [ChatGLM3](https://github.com/THUDM/ChatGLM3|
| **ChatGLM3-6B-base** | **第三代ChatGLM****基座模型****。**ChatGLM3-6B-Base 采用了更多样的训练数据、更充分的训练步数和更合理的训练策略。在语义、数学、推理、代码、知识等不同角度的数据集上测评显示，ChatGLM3-6B-Base 具有在 10B 以下的基础模型中最强的性能。 | 8K                | [Huggingface ](https://huggingface.co/THUDM/chatglm3-6b-base)\| [魔搭社区](https://modelscope.cn/models/ZhipuAI/chatglm3-6b-base) \| [始智社区](https://www.wisemodel.cn/models/ZhipuAI/chatglm3-6b-base) \| [Swanhub](https://swanhub.co/ZhipuAI/chatglm3-6b-base) \| [启智社区](https://openi.pcl.ac.cn/Zhipu.AI/ChatGLM3/modelmanage/model_readme_tmpl?name=chatglm3-6b-base) |                                                              |
| **ChatGLM3-6B-32k**  | **第三代ChatGLM长上下文对话模型。**在ChatGLM3-6B的基础上进一步强化了对于长文本的理解能力，能够更好的处理最多32K长度的上下文。 | 32K               | [Huggingface](https://huggingface.co/THUDM/chatglm3-6b-32k) \| [魔搭社区](https://modelscope.cn/models/ZhipuAI/chatglm3-6b-32k) \| [始智社区](https://www.wisemodel.cn/models/ZhipuAI/chatglm3-6b-32k) \| [Swanhub](https://swanhub.co/ZhipuAI/chatglm3-6b-32k) \| [启智社区](https://openi.pcl.ac.cn/Zhipu.AI/ChatGLM3/modelmanage/model_readme_tmpl?name=chatglm3-6b-32k) |                                                              |



# 3. ChatGLM3-6B本地部署流程

&emsp;&emsp;对于部署ChatGLM3-6B来说，从官方说明上看，其规定了Transformers 库版本应该 4.30.2 以及以上的版本 ，torch 库版本应为 2.0 及以上的版本，gradio 库版本应该为 3.x 的版本，以获得最佳的推理性能。所以为了保证 torch 的版本正确，建议大家严格按照官方文档的说明安装相应版本的依赖包。

- **Step 1. 创建conda虚拟环境**

&emsp;&emsp;Conda创建虚拟环境的意义在于提供了一个隔离的、独立的环境，用于Python项目和其依赖包的管理。每个虚拟环境都有自己的Python运行时和一组库。这意味着我们可以在不同的环境中安装不同版本的库而互不影响。例如，可以在一个环境中使用Python 3.8，而在另一个环境中使用Python 3.9。对于大模型来说，建议Python版本3.10以上。创建的方式也比较简单，使用以下命令创建一个新的虚拟环境：

```bash
# myenv 是你想要给环境的名称，python=3.8 指定了要安装的Python版本。你可以根据需要选择不同的名称和/或Python版本。

conda create -n chatglm3_test python=3.11
```

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/1.PNG" width=50%></div>

创建虚拟环境后需要激活它才能开始使用，使用以下命令来激活刚创建的虚拟环境：
```bash
conda activate chatglm3_test 
```

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/3.PNG" width=70%></div>

&emsp;&emsp;如果成功激活，可以看到在命令行的最前方的括号中，就标识了当前的虚拟环境（chatglm3_test），然后，按照官方的要求安装torch。

- **Step 2. 查看当前驱动最高支持的CUDA版本**

CUDA（Compute Unified Device Architecture，统一计算设备架构）是一种操作GPU的软件架构，它是一种通用并行计算架构，使GPU能够解决复杂的计算问题。

&emsp;&emsp;我们需要根据CUDA版本选择Pytorch框架，先看下当前的CUDA版本,通过以下命令实现功能：

```bash
nvidia-sml
```

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/4.PNG" width=70%></div>

可以看到本台机器的CUDA版本是12.2，接下来的库和依赖安装也要选择与其兼容的Pytorch版本。

- **Step 3. 在虚拟环境中安装Pytorch**

PyTorch 是一个开源的深度学习框架，由 Facebook 的人工智能研究团队（FAIR）开发和维护（当然现在可能也更名叫MAIR了）。Pytorch专门针对 GPU 加速的深度神经网络（DNN）编程。Torch 是一个经典的对多维矩阵数据进行操作的张量（tensor ）库，在机器学习和其他数学密集型应用有广泛应用。与Tensorflow的静态计算图不同，pytorch的计算图是动态的，可以根据计算需要实时改变计算图。并且Pytorch具有非常简洁的格式为广大Python用户舒适体验。



&emsp;&emsp;进入Pytorch官网：https://pytorch.org/get-started/previous-versions/

<div align=center><img src="https://muyu001.oss-cn-beijing.aliyuncs.com/img/image-20240103163206436.png" width=80%></div>

&emsp;&emsp;当前的电脑CUDA的最高版本要求是12.2，所以需要找到 >=12.2版本的Pytorch。

<div align=center><img src="https://snowball101.oss-cn-beijing.aliyuncs.com/img/202401041455184.png" width=80%></div>

&emsp;&emsp;直接复制对应的命令，进入终端执行即可。这实际上安装的是为 CUDA 12.1 优化的 PyTorch 版本。这个 PyTorch 版本预编译并打包了与 CUDA 12.1 版本相对应的二进制文件和库。

conda install pytorch==2.3.1 torchvision==0.18.1 torchaudio==2.3.1 pytorch-cuda=12.1 -c pytorch -c nvidia


<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/5.PNG" width=90%></div>

- **Step 4. 安装Pytorch验证**

&emsp;&emsp;安装完毕后，如果想要检查是否成功安装了GPU版本的PyTorch，可以通过几个简单的步骤在 Python 环境中进行验证：
```bash
import torch

print(torch.cuda.is_available())
```

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/7.PNG" width=80%></div>

&emsp;&emsp;如果输出是 True，则表示GPU版本的PyTorch已经安装成功并且可以使用CUDA，如果输出是 False，则表明没有安装GPU版本的PyTorch，或者CUDA环境没有正确配置，此时根据教程，重新检查自己的执行过程。

完成验证之后使用ctril+D退出python环境，继续命令行操作。

- **Step 5. 下载git及建立项目文件夹**

&emsp;&emsp;在执行命令之前，先安装git软件包。

```python
sudo apt install git
```

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/8.PNG" width=80%></div>

&emsp;&emsp;然后创建一个存放ChatGLM3-6B项目文件的文件夹。
```python
mkdir chatglm3
ls
cd chatglm3
ls
```

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/9.PNG" width=90%></div>

- **Step 6. 获取ChatGLM3-6B项目资源**

这里提供一种无需翻墙可以下载模型办法，到魔搭社区官网中快速下载，以下是model scope的ChatGLM3-6B的网址：https://www.modelscope.cn/models/ZhipuAI/chatglm3-6b

登录进入官网后点击模型库选项开始搜索所需要的模型名称。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20240818095223108.png" width=80%></div>

在模型搜索栏输入关键词GLM3-6B即可找到对应系列模型，点击标题名为chatglm3-6b的选项，进入模型详情页。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20240818095403561.png" width=80%></div>

在详情页有模型的基本介绍、对应官网的链接、以及它的下载资源和简单的下载流程说明。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20240818095443362.png" width=80%></div>

- **Step 7. 升级pip版本**

&emsp;&emsp;pip 是 Python 的一个包管理器，用于安装和管理 Python 软件包。允许从 Python Package Index（PyPI）和其他索引中安装和管理第三方库和依赖。一般使用 pip 来安装、升级和删除 Python 软件包。除此之外，pip 自动处理 Python 软件包的依赖关系，确保所有必需的库都被安装。在Python环境中，尽管我们是使用conda来管理虚拟环境，但conda是兼容pip环境的，所以使用pip下载必要的包是完全可以的。

&emsp;&emsp;在终端命令行升级 pip，确保使用的是最新版本的 pip，如果使用的是旧版本的 pip，可能无法安装一些最新的包，或者可能无法正确解析依赖关系。最新版本的pip这样可以避免在安装库时出现兼容性问题。
```
python -m pip install --upgrade pip
```

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/16.png" width=100%></div>

- **Step 8. 使用pip安装相关的软件依赖**

首先，我们需要安装一系列的 Python 包，这些包是使用 Hugging Face Transformers 库所必需的。可以使用以下命令来安装这些依赖项：
```
pip install protobuf 'transformers>=4.30.2' cpm_kernels 'torch>=2.0' gradio mdtex2html sentencepiece accelerate
```


<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20240827113838550.png" width=80%></div>

随后进行modelscope的库下载，当程序运行完成并在底部发现Succefully installed modelscope字样时说明下载成功。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/13.png" width=80%></div>

- **Step 9. 在Python环境下载模型**

输入python进入环境，输入以下代码使用snapshot_download函数进行模型的下载：
```python
from modelscope import snapshot_download
model_dir = snapshot_download("ZhipuAI/chatglm3-6b", revision = "v1.0.0")
```

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/14.png" width=100%></div>

transformers版本过高或过低都会导致运行大模型时出现错误，因此我们需要指定transformers的版本使得版本兼容，通过以下指令可以查看transformers的版本：
```
pip show transformers
```

通过以下指令来卸载当前的版本以及安装指定的transformers版本
```
pip uninstall transfomers
pip install --upgrade transformers==4.37.2

```

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/17.png" width=80%></div>

- **Step 10. 在Python环境下载模型**

在python环境中继续输入以下代码调用 ChatGLM3-6B 模型来生成对话：
```python
from modelscope import AutoTokenizer, AutoModel, snapshot_download
model_dir = snapshot_download("ZhipuAI/chatglm3-6b", revision = "v1.0.0")
tokenizer = AutoTokenizer.from_pretrained(model_dir, trust_remote_code=True)
model = AutoModel.from_pretrained(model_dir, trust_remote_code=True).half().cuda()
model = model.eval()
response, history = model.chat(tokenizer, "你好", history=[])
print(response)
response, history = model.chat(tokenizer, "晚上睡不着应该怎么办", history=history)
print(response)
```


<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/18.png" width=100%></div>

大模型推理完毕后可以看到返回的结果，说明安装部署成功。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20240818094635045.png" width=100%></div>

以上就是Ubuntu系统下通过python环境在终端安装部署ChatGLM3-6B模型的全部流程，欢迎大家进行学习交流！